In [20]:
import xarray as xr # Basic package to deal with data arrays
import xarray_sentinel # Handles most basic SAR operations
import matplotlib as plt
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

In [39]:
zarr_paths = sorted(glob.glob("/Users/devseed/Documents/repos/eopf-cpm/02_zarr_converted/*.zarr"))
datatrees = [xr.open_datatree(path, consolidated=True) for path in zarr_paths]
datasets = [xr.open_dataset(path, consolidated=True) for path in zarr_paths]

In [128]:
product_names =[]
for i in datatrees:
    product_names.append(i)
    print(i.groups[1])

/S01SIWGRD_20241007T180256_0025_A320_D46B_06D943_VH
/S01SIWGRD_20241019T180256_0025_A321_2D52_06E02E_VH
/S01SIWGRD_20241031T180256_0025_A322_479F_06E71E_VH
/S01SIWGRD_20241112T180255_0025_A323_DC29_06EE16_VH
/S01SIWGRD_20241124T180254_0025_A324_BA27_06F516_VH
/S01SIWGRD_20241206T180253_0025_A325_25AD_06FBFD_VH
/S01SIWGRD_20241218T180252_0025_A326_0BC2_0702F2_VH
/S01SIWGRD_20241230T180251_0025_A327_15AC_0709DD_VH
/S01SIWGRD_20250111T180250_0025_A328_ADBB_0710C7_VH
/S01SIWGRD_20250123T180249_0025_A329_A784_0717B9_VH
/S01SIWGRD_20250204T180249_0025_A330_4373_071EA2_VH
/S01SIWGRD_20250216T180248_0025_A331_8AC7_0725AE_VH
/S01SIWGRD_20250228T180248_0025_A332_D056_072CD1_VH
/S01SIWGRD_20250312T180248_0025_A333_4F5B_0733E6_VH
/S01SIWGRD_20250324T180248_0025_A334_04B7_073AD0_VH
/S01SIWGRD_20250405T180249_0025_A335_248B_0741EA_VH
/S01SIWGRD_20250417T180249_0025_A336_F868_07490E_VH
/S01SIWGRD_20250429T180249_0025_A337_10EA_07503A_VH
/S01SIWGRD_20250511T180249_0025_A338_E613_0756D0_VH


In [179]:
antes = product_names[2].groups[4]#.[product_names[1].groups[0]]
print(str(antes))

/S01SIWGRD_20241031T180256_0025_A322_479F_06E71E_VH/measurements


In [180]:
measurements = datatrees[(antes)].to_dataset()

TypeError: list indices must be integers or slices, not str

In [152]:
#measurements_paths = [f"{name}/measurements" for name in product_names]
#for i in measurements_paths:
#    print(i)